In [1]:
from sympy import diff, Symbol, sin, tan
import sympy
import numpy as np
import math
import cv2
from fractions import Fraction
from sympy import diff, Symbol
from sympy import *
def Distance(x1,y1,z1,x2,y2,z2):
    dist = sympy.sqrt((x2 - x1)**2 + (y2 - y1)**2+(z2 - z1)**2)
    return dist

        
# construct the argument parse and parse the arguments
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5, 5))
# load the image
image = cv2.imread('Long.png')
#image = cv2.resize(image, (1000, 750))
#cv2.imshow("images", image)
# define the list of boundaries
thres = 40
boundaries = [
	([0, 0, 230], [50, 50, 255]),#red
    ([0, 100, 230], [30, 180, 255]),#orange
    ([230, 0, 0], [255, 50,50 ]) # blue    
]
clone = image.copy()
# loop over the boundaries
ax=[]
ay=[]
for (lower, upper) in boundaries:
    lower = np.array(lower, dtype = "uint8")
    upper = np.array(upper, dtype = "uint8")
	# find the colors within the specified boundaries and apply
	# the mas
    mask = cv2.inRange(image, lower, upper)
    output1 = cv2.bitwise_and(image, image, mask = mask)
    x = output1.nonzero()
    ax.append(x[1][0])
    ay.append(x[0][0])
x_center=[]
y_center=[]
f=4.73#mm

for i in ax:
    x_center.append(i-2000)
    #x_center.append(i)
for i in ay:
    y_center.append(i-1500)
    #y_center.append(i)
print("x_center",x_center)
print("y_center",y_center)
X1,Y1,Z1=0,0,0
X2,Y2,Z2=27.5,5.8,0
X3,Y3,Z3=21,-8.3,0
x1,y1=x_center[0],y_center[0]
x2,y2=x_center[1],y_center[1]
x3,y3=x_center[2],y_center[2]

ratio=Distance(X1,Y1,Z1,X2,Y2,Z2)/(Distance(x_center[0],y_center[0],0,x_center[1],y_center[1],0)*1.6*0.0001)#cm
print("ratio",int(ratio))
S2real=f*0.1*ratio#cm
for i in range(2):
    S = symbols('S')
    S=solve((1/S)+(1/S2real)-(1/f),S)[0]
    #print("S",S)
    S2real = symbols('S2real')
    S2real=solve((1/S)+(1/S2real)-(1/f),S2real)[0]
print("S2real",S2real)#cm   
print("S",S)#cm

x_center [-313, 477, 270]
y_center [342, 211, 607]
ratio 219
S2real 103.754526306742
S 4.95593291616157


In [ ]:
def Distance(x1,y1,z1,x2,y2,z2):
    dist = sympy.sqrt((x2 - x1)**2 + (y2 - y1)**2+(z2 - z1)**2)
    return dist

#print(Distance(100,100,10,100,100,30))

camera_real_x=((0-x1)*1.6*0.0001*ratio)
camera_real_y=((0-y1)*1.6*0.0001*ratio)
camera_real_z=S2real#initial pose for camera cm
A = Symbol('A',real=True)
B = Symbol('B',real=True)
C = Symbol('C',real=True)
angle1=sympy.acos(np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/(Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2)))
angle2=sympy.acos(np.dot((x1-A,y1-B,0-C),(x2-A,y2-B,0-C))/(Distance(A,B,C,x1,y1,0)*Distance(A,B,C,x2,y2,0)))
angle3=sympy.acos(np.dot((X1-A,Y1-B,Z1-C),(X3-A,Y3-B,Z3-C))/(Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X3,Y3,Z3)))
angle4=sympy.acos(np.dot((x1-A,y1-B,0-C),(x3-A,y3-B,0-C))/(Distance(A,B,C,x1,y1,0)*Distance(A,B,C,x3,y3,0)))

delt_A=diff((angle1-angle2), A)
delt_B=diff((angle1-angle2), B)
delt_C=diff((angle1-angle2), C)

delt_A_2=diff((angle3-angle4), A)
delt_B_2=diff((angle3-angle4), B)
delt_C_2=diff((angle3-angle4), C)

for i in range(5000):
    delt_A=delt_A.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_B=delt_B.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_C=delt_C.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_A_2=delt_A.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_B_2=delt_B.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_C_2=delt_C.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    camera_real_x=camera_real_x-5*delt_A-5*delt_A_2
    camera_real_y=camera_real_y-5*delt_B-5*delt_B_2
    camera_real_z=camera_real_x-1*delt_C-1*delt_C_2
    #print(delt_A)
    #print(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2))<1)
    angle1=sympy.acos(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)))
    angle2=sympy.acos(np.dot((x1-camera_real_x,y1-camera_real_y,108-camera_real_z),(x2-camera_real_x,x2-camera_real_y,108-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,x1,y1,0)*Distance(camera_real_x,camera_real_y,camera_real_z,x2,y2,0)))
    print("x=%f y=%f z=%f"%(camera_real_x,camera_real_y,camera_real_z))
    #print(angle1-angle2)
    #print("angle1=%f angle2=%f"%(angle1,angle2))
    if abs(angle1-angle2)<0.1:
        print("converge x=%f y=%f z=%f"%(camera_real_x,camera_real_y,camera_real_z))
        break
    

x=10.989754 y=-11.978396 z=103.754526
x=10.994252 y=-11.953732 z=103.754526
x=10.998749 y=-11.929068 z=103.754526
x=11.003246 y=-11.904404 z=103.754526
x=11.007743 y=-11.879739 z=103.754526
x=11.012240 y=-11.855075 z=103.754526
x=11.016738 y=-11.830411 z=103.754526
x=11.021235 y=-11.805746 z=103.754526
x=11.025732 y=-11.781082 z=103.754526
x=11.030229 y=-11.756418 z=103.754526
x=11.034726 y=-11.731754 z=103.754526
x=11.039224 y=-11.707089 z=103.754526
x=11.043721 y=-11.682425 z=103.754526
x=11.048218 y=-11.657761 z=103.754526
x=11.052715 y=-11.633096 z=103.754526
x=11.057212 y=-11.608432 z=103.754526
x=11.061710 y=-11.583768 z=103.754526
x=11.066207 y=-11.559104 z=103.754526
x=11.070704 y=-11.534439 z=103.754526
x=11.075201 y=-11.509775 z=103.754526
x=11.079698 y=-11.485111 z=103.754526
x=11.084196 y=-11.460446 z=103.754526
x=11.088693 y=-11.435782 z=103.754526
x=11.093190 y=-11.411118 z=103.754526
x=11.097687 y=-11.386454 z=103.754526
x=11.102184 y=-11.361789 z=103.754526
x=11.106682 

In [ ]:
print("apple")